<a href="https://colab.research.google.com/github/mlaricobar/WebScrapingCourse/blob/master/DMC2019I_NOT1_Introducci%C3%B3n_API_Facebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploración del Graph API de Facebook

![alt text](https://cdn-images-1.medium.com/max/1600/1*o5I76UNWVmvrfY03fYLfRw.jpeg)

Practiquemos con nuestros datos de Facebook. En esta oportunidad, haremos uso de una de las herramientas de Facebook llamado **Facebook Graph** con el fin de extraer los datos generados por el uso en su Plataforma.

Previo a su uso, necesitamos tener instalado las librerías json, requests y pandas.

### Graph API de Facebook

El Explorador de API Graph es una herramienta que te permite construir y realizar consultas a la API Graph y ver sus respuestas, para cualquier aplicación en la que tenga un rol de administrador, desarrollador o tester.

La herramienta es extremadamente útil durante el desarrollo de aplicaciones porque hereda todas las configuraciones de su aplicación, incluidos los permisos de inicio de sesión aprobados, las funciones y la configuración de los productos que haya agregado.

Puedes probarlo en el siguiente enlace: [API Graph de Facebook](https://developers.facebook.com/tools/explorer/?classic=0)

<img src="https://github.com/mlaricobar/Machine-Learning-Course/blob/master/images/facebook-graph.png?raw=1" width="700">

### Arquitectura Básica de un API

A continuación, se muestra el diagrama de Arquitectura de nuestra solución


![alt text](https://github.com/mikolarico/FS_Machine_Learning_Python/blob/master/api-solution-architecture.png?raw=true)

Este diagrama muestra de una manera simple como funciona un API, esto con el fin de entender cómo podemos acceder a los datos de Facebook a través de su Graph API.

Así mismo, se muestra un ejemplo del uso del Graph de Facebook para extraer algunos datos de nuestro Perfil Público.

In [0]:
# Importamos los packages que vamos a necesitar

## El paquete o package Json nos permitirá trabajar con estructuras de datos del tipo JSON
import json
## El paquete o package Requests nos permitirá hacer peticiones del tipo HTTPS a un Servidor Web
import requests
## El paquete o package Pandas nos permitirá manipular y analizar datos estructurados
import pandas as pd

Debemos utilizar el identificador de Facebook y los tokens que reflejan los permisos de acceso a los datos de Facebook (por ejemplo: likes, tagged places, etc) 

In [0]:
id_facebook = "<--- ESCRIBE TU IDENTIFICADOR DE FACEBOOK --->"
token = "<--- ESCRIBE TU TOKEN GENERADO --->"

Definición de url que utilizaremos para hacer la solicitud al Graph API de Facebook

In [0]:
host = "https://graph.facebook.com/v3.2/"
fields = "id,email,first_name,last_name,name,birthday,age_range,gender,location{general_info,location},languages{name,description},picture.width(500).height(500)"
url = "{host}{id}?fields={fields}&access_token={token}".format(host=host, id=id_facebook, fields=fields, token=token)

In [0]:
print(url)

In [0]:
# Utilizamos el método get del package Requests para hacer la solicitud
response = requests.get(url)

In [0]:
print("Respuesta a la solicitud: {0}".format(response))
print("Tipo de dato de la respuesta: {0}".format(type(response)))

In [0]:
# Usar el método .json() para obtener los datos de la respuesta
response.json()

In [0]:
# Usamos del método dumps del package Json para dar un mejor formato al resultado
print(json.dumps(response.json(), indent=4))

### Perfil Público


In [0]:
fields = 'id,email,first_name,last_name,name,birthday'
url = "{host}{id}?fields={fields}&access_token={token}".format(host=host, id=id_facebook, fields=fields, token=token)

In [0]:
print(url)

In [0]:
response = requests.get(url)
perfil = response.json()

In [0]:
print(json.dumps(perfil, indent=4))

¿Cómo lo puedo tranformar a un DataFrame?

In [0]:
dfPerfil = pd.DataFrame.from_dict(perfil, orient="index").T

In [0]:
dfPerfil.rename(columns={"id": "id_user"}, inplace=True)

In [0]:
dfPerfil.head()

In [0]:
# Guardamos el Identificador del Usuario como variable para utilizarlo con las demás fuente (Likes y Tagged Places)
idUser = perfil["id"]

### Comparte tu información para un análisis colectivo

In [0]:
# Instalaremos el package pydocumentdb para poder interactura con la Base de Datos de Azure Cosmos DB
!pip install pydocumentdb

In [0]:
import pydocumentdb
from pydocumentdb import documents
from pydocumentdb import document_client

In [0]:
def upload_information(df, collection_name):
  ENDPOINT = "https://mlaricobar.documents.azure.com:443/"
  MASTERKEY = "wGUpP07DsObsE85fNeb8kywcV8Y1gYFt10DUR4wWjJuQI9LrjCWtWwG2cGMGrMfhNShIUKDwJqD1MwZGZ34APA=="

  connectionPolicy = documents.ConnectionPolicy()
  connectionPolicy.EnableEndpointDiscovery
  connectionPolicy.PreferredLocations = ["East US 2"]
  client = document_client.DocumentClient(ENDPOINT, {'masterKey': MASTERKEY}, connectionPolicy)
  
  database = 'WebScrapingCourse01'
  dblink = 'dbs/' + database
  collink = dblink + '/colls/' + collection_name
  
  log_upload = [client.CreateDocument(collink, d) for d in df.to_dict(orient="records")]
  
  return log_upload

In [0]:
log_upload = upload_information(dfPerfil, "UserProfile")

In [0]:
log_upload

### Likes

Ahora, haremos uso de Graph de Facebook para extraer nuestros Likes.

In [0]:
fields = "likes.limit(100){id,name,description,created_time,link,category,about,fan_count}"
url = "{host}{id}?fields={fields}&access_token={token}".format(host=host, id=id_facebook, fields=fields, token=token)

In [0]:
response = requests.get(url)
likes = response.json()

In [0]:
print(json.dumps(likes, indent=4))

In [0]:
likes = likes["likes"]

Nos damos cuenta de que a diferencia del resultado obtenido en el Perfil, dentro del diccionario resultado de la solicitud de likes se encuentra un elemento llamado paging que contiene la dirección url de la siguiente paginación.

In [0]:
l = []
i = 1
current_url_pagging, next_url_pagging = "", " "
while current_url_pagging != next_url_pagging:
    try:
        if 'data' in likes.keys():
            l += likes['data']
            print("Petición #{0}".format(i))
            print("\t" + likes['paging']['next'])
            i+=1
            current_url_pagging = likes['paging']['next']
            likes = requests.get(current_url_pagging).json()
            print("\t {0} nuevos Likes".format(len(likes['data'])))
            next_url_pagging = likes['paging']['next']
        else:
          break
    except KeyError:
        break

In [0]:
print("Se han extraído {0} likes".format(len(l)))
print("Un ejemplo de un registro de Like: \n")
print(json.dumps(l[0], indent=4))

A partir de la lista generada de likes, crea un DataFrame llamado `dfLikes` y ordena las columnas de la siguiente manera: `id, name, created_time, link, description, category y fan_count`.

In [0]:
dfLikes = pd.DataFrame(l)[["id", "name", "created_time", "link", "description", "category", "fan_count"]]

In [0]:
# Eliminamos posibles duplicados,
print(dfLikes.shape)
dfLikes = dfLikes.drop_duplicates(subset=["id"]).reset_index(drop=True)
print(dfLikes.shape)

In [0]:
dfLikes.head()

Demos un vistazo general al DataFrame para saber cuántos registros y columnas tiene, cuánto pesa, qué tipos de datos contiene y la cantidad de nulos por columna.

In [0]:
dfLikes.info()

Si nos damos cuenta, la columna **created_time** es del tipo `Object`. Necesitamos crear una nueva columna que tenga el formato "YYYYMMDD".

In [0]:
# importar el módulo datetime del package datetime
from datetime import datetime

In [0]:
# Eliminar los registros que tengan nulos en la columna "created_time"
dfLikes.dropna(subset=["created_time"], inplace=True)

In [0]:
# Crear la nueva columna usando funciones de la librería datetime
# El método strptime(value, format) transforma una cadena de texto a un tipo de dato Datetime indicandole un formato de fecha, y el método hace lo contrario, es decir a partir de un valor del tipo Datetime lo transforma a una cadena de texto con un formato definido
dfLikes["date"] = dfLikes["created_time"].apply(lambda d: datetime.strptime(d, '%Y-%m-%dT%H:%M:%S+%f').strftime('%Y%m%d'))

Ordena el DataFrame `dfLikes` por el nuevo campo **date** de forma ascendente.

In [0]:
dfLikes.sort_values(by="date", ascending=True, inplace=True)

¿Cómo ha sido mi comportamiento diario dando likes? Es decir, ¿Cuántos likes he dado en cada día?

In [0]:
# Agrupar por día y contar la cantidad de páginas a las que les dí likes
dfLikesDateAgg = dfLikes.groupby("date", as_index=False).agg({"id": "count"})
dfLikesDateAgg

In [0]:
g = dfLikesDateAgg.plot(x="date", y="id", kind="line", figsize=(30, 5))

¿Cómo ha sido mi comportamiento mensual dando likes? Es decir, ¿Cuántos likes he dado en cada mes?

In [0]:
# Crear la nueva columna usando funciones de la librería datetime
dfLikes["periodo"] = dfLikes["created_time"].apply(lambda d: datetime.strptime(d, '%Y-%m-%dT%H:%M:%S+%f').strftime('%Y%m'))

In [0]:
# Agrupar por periodo y contar la cantidad de páginas a las que les dí likes
dfLikesMonthAgg = dfLikes.groupby("periodo", as_index=False).agg({"id": "count"})
dfLikesMonthAgg

In [0]:
g = dfLikesMonthAgg.plot(x="periodo", y="id", kind="line", figsize=(30, 5))

In [0]:
# Crear la nueva columna usando funciones de la librería datetime
dfLikes["año"] = dfLikes["created_time"].apply(lambda d: datetime.strptime(d, '%Y-%m-%dT%H:%M:%S+%f').strftime('%Y'))

In [0]:
# Agrupar por Año y contar la cantidad de páginas a las que les dí likes
dfLikesYearAgg = dfLikes.groupby("año", as_index=False).agg({"id": "count"})
dfLikesYearAgg

In [0]:
g = dfLikesYearAgg.plot(x="año", y="id", kind="line", figsize=(30, 5))

In [0]:
# Agrupar por año y obtener el promedio de la cantidad de fans de las páginas a las que les dí likes
dfLikes.groupby("año").agg({"fan_count": "mean"}).astype(int)

Ordenemos los registros en base a la columna created_time

In [0]:
dfLikes.sort_values(by="date", ascending=True, inplace=True)

Agrupemos los likes por categoría, y ordenarlos de forma descendente en base a la cantidad de likes.

In [0]:
dfLikes.groupby("category", as_index=False).agg({"id": "count"})

Crear una Pivot Table, donde pueda observar la cantidad de páginas por año (columnas) y categoría (índices).

In [0]:
dfLikesPivot = dfLikes.pivot_table(index="category", columns=["año"], values="id", aggfunc="count", fill_value=0, margins=True)

En base a la tabla anterior, identificar cuántos likes he dado sólo a páginas de la categoría **Education**.

In [0]:
dfLikesPivot.reset_index(inplace=True)
dfLikesPivot.loc[dfLikesPivot["category"] == "Education"]

### Comparte tu información para un análisis colectivo

In [0]:
dfLikes["id_user"] = idUser
dfLikes.rename(columns={"id": "id_like"}, inplace=True)

In [0]:
dfLikes.info()

In [0]:
dfLikes["description"] = dfLikes["description"].fillna("")

In [0]:
log_upload = upload_information(dfLikes, "UserLikes")

In [0]:
log_upload[0]

### Tagged Places

Ahora, haremos uso de Graph de Facebook para extraer nuestras Etiquetas.

In [0]:
fields = "tagged_places.limit(100){id,created_time,place}"
url = "{host}{id}?fields={fields}&access_token={token}".format(host=host, id=id_facebook, fields=fields, token=token)

In [0]:
response = requests.get(url)
tagged_places = response.json()

In [0]:
print(json.dumps(tagged_places, indent=4))

In [0]:
tagged_places = tagged_places["tagged_places"]

In [0]:
l = []
i = 0
current_url_pagging, next_url_pagging = "", " "
while current_url_pagging != next_url_pagging:
    try:
        if 'data' in tagged_places.keys():
            l += tagged_places['data']
            print("Petición #{0}".format(i))
            print("\t" + tagged_places['paging']['next'])
            i+=1
            current_url_pagging = tagged_places['paging']['next']
            tagged_places = requests.get(current_url_pagging).json()
            print("\t {0} nuevos Likes".format(len(tagged_places['data'])))
            next_url_pagging = tagged_places['paging']['next']
        else:
          break
    except KeyError:
        break

In [0]:
print("Se han extraído {0} etiquetas".format(len(l)))
print("Un ejemplo de un registro de Etiqueta: \n")
print(json.dumps(l[0], indent=4))

A partir de la lista generada de etiquetas, crea un DataFrame llamado `dfEtiquetas` y ordena las columnas de la siguiente manera: `id, created_time y place`.

In [0]:
dfEtiquetas = pd.DataFrame(l)[["id", "created_time", "place"]]

Demos un vistazo general al DataFrame para saber cuántos registros y columnas tiene, cuánto pesa, qué tipos de datos contiene y la cantidad de nulos por columna.

In [0]:
dfEtiquetas.info()

Si nos damos cuenta, la columna **created_time** es del tipo `Object`. Necesitamos crear una nueva columna que tenga el formato "YYYYMMDD".

In [0]:
# Eliminar posibles duplicados
dfEtiquetas.drop_duplicates(subset=["id"], inplace=True)
dfEtiquetas.reset_index(drop=True, inplace=True)

In [0]:
# Eliminar los registros que tengan nulos en la columna "created_time"
dfEtiquetas.dropna(subset=["created_time"], inplace=True)

In [0]:
# Crear la nueva columna usando funciones de la librería datetime
dfEtiquetas["date"] = dfEtiquetas["created_time"].apply(lambda d: datetime.strptime(d, '%Y-%m-%dT%H:%M:%S+%f').strftime('%Y%m%d'))

In [0]:
# Crear 2 nuevas columnas llamadas longitude y latitude, basados en el campo place
dfEtiquetas["latitude"] = dfEtiquetas["place"].apply(lambda p: p["location"]["latitude"])
dfEtiquetas["longitude"] = dfEtiquetas["place"].apply(lambda p: p["location"]["longitude"])

In [0]:
# Crear una nueva columna con el nombre del Lugar
dfEtiquetas["name"] = dfEtiquetas["place"].apply(lambda p: p["name"])

In [0]:
dfEtiquetas

In [0]:
# Resumen dle DataFrame
dfEtiquetas.info()

In [0]:
# ¿En cuántas ciudades he sido etiquetado?... Y ¿Cuántas veces he sido etiquetado en cada ciudad?
dfEtiquetas["city"] = dfEtiquetas["place"].apply(lambda p: p["location"]["city"] if "city" in p["location"].keys() else None)

In [0]:
dfEtiquetas.groupby("city", as_index=False).agg({"id": "count"}).sort_values(by="id", ascending=False)

# Visualizar los lugares donde me etiquetaron en un Mapa

Para poder visualizar las coordenadas de todos los lugares donde hemos sido taggeados extraídos por la solicitud al Graph API de Facebook, haremos uso de [Mapbox](https://www.mapbox.com/) y [Plotly](https://plot.ly/feed/#/).

Mapbox es un proveedor de mapas on-line realizados por encargo para páginas webs como **Foursquare, Pinterest, Evernote, Financial Times, EThe Weather Channel y Uber Tecnologías**. Desde 2010, ha expandido rápidamente su nicho de mapas como respuesta a la limitada elección que ofrecen otros proveedores como Google Maps y OpenStreetMap.

<br>
<img src="https://github.com/mlaricobar/Machine-Learning-Course/raw/84d066f02eb8a3b0c342c2c1ad182657dc07f972/images/mapbox.png" width="400">

Para poder usar mapbox, necesitamos de un token que se genera al crear una cuenta.

Para poder usar esta herramienta dentro de Jupyter, debemos instalar el módulo [plotly](https://plot.ly/python/getting-started/).

In [0]:
!pip install plotly

In [0]:
# Importar los métodos del package Plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly

In [0]:
username_plotly = "<--- ESCRIBE TU USUARIO DE PLOTLY --->"
api_key_plotly = "<--- ESCRIBE EL API KEY GENERADO --->"
mapbox_access_token = "<--- ESCRIBE EL TOKEN GENERADO EN MAPBOX --->"

In [0]:
# Definir credenciales de Plotly y token de Mapbox
plotly.tools.set_credentials_file(username=username_plotly, api_key=api_key_plotly)

In [0]:
site_lat = dfEtiquetas.latitude
site_lon = dfEtiquetas.longitude
locations_name = dfEtiquetas.name

In [0]:
# Mostrar los puntos en el mapa
data = [
    go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=17,
            color='rgb(255, 0, 0)',
            opacity=0.7
        ),
        text=locations_name,
        hoverinfo='text'
    ),
    go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=8,
            color='rgb(242, 177, 172)',
            opacity=0.7
        ),
        hoverinfo='none'
    )]
        
layout = go.Layout(
    title='Lugares donde he sido Etiquetado',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=-12.047290,
            lon=-77.043427
        ),
        pitch=0,
        zoom=11,
        style= 'mapbox://styles/mapbox/light-v9'
    ),
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Lugares donde he sido Etiquetado')

### Comparte tu información para un análisis colectivo

In [0]:
dfEtiquetas["id_user"] = idUser
dfEtiquetas.rename(columns={"id": "id_tagged_place"}, inplace=True)

In [0]:
tagged_columns = ["id_user", "id_tagged_place", "name", "latitude", "longitude", "created_time", "date"]

In [0]:
log_upload = upload_information(dfEtiquetas[tagged_columns], "UserTaggedPlaces")

In [0]:
log_upload[0]

# Análisis Colectivo

In [0]:
def download_information(collection_name):
  ENDPOINT = "https://mlaricobar.documents.azure.com:443/"
  MASTERKEY = "wGUpP07DsObsE85fNeb8kywcV8Y1gYFt10DUR4wWjJuQI9LrjCWtWwG2cGMGrMfhNShIUKDwJqD1MwZGZ34APA=="

  connectionPolicy = documents.ConnectionPolicy()
  connectionPolicy.EnableEndpointDiscovery
  connectionPolicy.PreferredLocations = ["East US 2"]
  client = document_client.DocumentClient(ENDPOINT, {'masterKey': MASTERKEY}, connectionPolicy)
  
  database = 'WebScrapingCourse01'
  dblink = 'dbs/' + database
  collink = dblink + '/colls/' + collection_name
  
  result = pd.DataFrame(list(client.ReadDocuments(collink)))
  result = result[[c for c in result.columns if c not in ['_attachments', '_etag', '_rid', '_self', '_ts']]]
  
  return result

In [0]:
df_user_profile = download_information("UserProfile")
df_user_likes = download_information("UserLikes")
df_user_tagged_places = download_information("UserTaggedPlaces")

### 1. ¿Cuantos usuarios de Facebook somos?

In [0]:
df_user_profile["id_user"].nunique()

### 2. ¿Cual es la distribución de edades?

In [0]:
from dateutil.relativedelta import relativedelta

In [0]:
def birthday(date):
    # Get the current date
    now = datetime.utcnow()
    now = now.date()

    # Get the difference between the current date and the birthday
    age = relativedelta(now, datetime.strptime(date, '%d/%M/%Y'))
    age = age.years

    return age

In [0]:
df_user_profile["age"] = df_user_profile["birthday"].apply(birthday)

In [0]:
g = df_user_profile["age"].hist()

### 3. ¿Quiénes son los más activos en Redes? (Top 5 de usuarios con más Likes)

In [0]:
df_user_likes_agg = df_user_likes.groupby("id_user", as_index=False).agg({"id": "nunique"})
df_user_likes_agg.sort_values(by="id", ascending=False).merge(df_user_profile[["name", "id_user"]], how="left", on="id_user").head(5)

### 4. ¿Quiénes son los más etiquetados? (Top 5 con más Tags)

In [0]:
df_user_tagged_places_agg = df_user_tagged_places.groupby("id_user", as_index=False).agg({"id": "nunique"})
df_user_tagged_places_agg.sort_values(by="id", ascending=False).merge(df_user_profile[["name", "id_user"]], how="left", on="id_user").head(5)

### 5. ¿Página con más likes?

In [0]:
df_user_likes_page_agg = df_user_likes.groupby("name", as_index=False).agg({"id_user": "nunique"}).sort_values(by="id_user", ascending=False)
df_user_likes_page_agg.head(5)

### 5. ¿Categoría de página con más likes?

In [0]:
df_user_likes_page_category_agg = df_user_likes.groupby("category", as_index=False).agg({"id_user": "count"}).sort_values(by="id_user", ascending=False)
df_user_likes_page_category_agg.head(5)

FIN DE NOTEBOOK